In [102]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import scan
from bluesky.plan_stubs import mv, open_run, close_run, create, \
    save, drop, rd, trigger, trigger_and_read, declare_stream
from ophyd.sim import det1, noisy_det, motor1
from databroker import Broker
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from IPython.display import display
install_nb_kicker()

ion()
RE = RunEngine()

db = Broker.named('temp')
RE.subscribe(db.insert)
bec = BestEffortCallback()
RE.subscribe(bec)

1

## Documents

Um dos principais objetivos do Bluesky é habilitar a coleta de dados e metadados. A meneira que ele faz isso é através dos **Documentos**nts.

Documentos são basicamente dicionários em Python com um esquema definido, ou seja, é um dicionário organizado de uma maneira específica e documentada. Esses **documentos** são gerados pela RunEngine durante a execução de planos e todos os dados e metadados estão organizados dentro dos **Documentos**.

### Runs e eventos

Planos podem gerar dados e metadados, porém como é possível organizar ou agrupar informações em Eventos e Runs?

In [45]:
def one_run_one_event():
    yield from open_run()
    yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [46]:
RE(one_run_one_event())



Transient Scan ID: 7     Time: 2024-09-10 10:26:20
Persistent Unique Scan ID: '0bb92f4f-11e7-4b50-a4e6-951b8e3afa37'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |       det1 |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:26:20.8 |      0.000 |      0.937 |
+-----------+------------+------------+------------+
generator one_run_one_event ['0bb92f4f'] (scan num: 7)





('0bb92f4f-11e7-4b50-a4e6-951b8e3afa37',)

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Uma linha (um evento)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [48]:
def one_run_n_events():
    yield from open_run()
    for i in range(4):
        yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [49]:
RE(one_run_n_events())



Transient Scan ID: 8     Time: 2024-09-10 10:28:05
Persistent Unique Scan ID: '25f47d38-0437-4744-931e-648050894f41'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |       det1 |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:28:05.9 |      0.000 |      1.028 |
|         2 | 10:28:05.9 |      0.000 |      0.997 |
|         3 | 10:28:05.9 |      0.000 |      0.974 |
|         4 | 10:28:05.9 |      0.000 |      0.975 |
+-----------+------------+------------+------------+
generator one_run_n_events ['25f47d38'] (scan num: 8)





('25f47d38-0437-4744-931e-648050894f41',)

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Quatro linhas (quatro eventos)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [50]:
def multi_run():
    for i in range(2):
        yield from one_run_n_events()

In [52]:
RE(multi_run())



Transient Scan ID: 9     Time: 2024-09-10 10:29:25
Persistent Unique Scan ID: '132f12bf-6bb0-47d8-b552-d951d04274d4'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |       det1 |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:29:25.1 |      0.000 |      1.028 |
|         2 | 10:29:25.1 |      0.000 |      0.933 |
|         3 | 10:29:25.1 |      0.000 |      1.088 |
|         4 | 10:29:25.1 |      0.000 |      0.922 |
+-----------+------------+------------+------------+
generator multi_run ['132f12bf'] (scan num: 9)





Transient Scan ID: 10     Time: 2024-09-10 10:29:25
Persistent Unique Scan ID: '2412d7e8-f48c-4de4-a1ed-925adfc0867a'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |       det1 |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:29:25.2 |      0.000 |      1.061 |
|         2 | 10:29:25.2 |     

('132f12bf-6bb0-47d8-b552-d951d04274d4',
 '2412d7e8-f48c-4de4-a1ed-925adfc0867a')

Neste exemplo temos:
 - Duas tabelas (duas Runs)
 - Quatro linhas (quatro eventos por Run)
 - Duas colunas (além de seq_num e time), representando os dois detectores

   
Além disso, agora o retorno da função é uma tupla com multiplos uids, cada um relacionado com uma Run

A definição de uma Run é flexível, pois tudo depende de como você gostaria de organizar seus dados, seja para coleta ou análise.

### Documentos de uma Run

#### O Start Document
O Start document contém informações de metadados no início da run:
- time - Tempo de início do plano
- plan_name - Nome do plano
- uid - ID único de identificação da Run
- scan_id - id "mais amigável", porém não é necessáriamente único
- metadados

In [56]:
uid, = RE(scan([det1], motor1, 0, 10, 6))



Transient Scan ID: 12     Time: 2024-09-10 10:40:58
Persistent Unique Scan ID: 'ec1f89aa-e10f-463e-8d91-e97e0a9e76f7'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |     motor1 |       det1 |
+-----------+------------+------------+------------+
|         1 | 10:40:58.2 |      0.000 |      5.000 |
|         2 | 10:40:58.2 |      2.000 |      0.002 |
|         3 | 10:40:58.3 |      4.000 |      0.000 |
|         4 | 10:40:58.3 |      6.000 |      0.000 |
|         5 | 10:40:58.3 |      8.000 |      0.000 |
|         6 | 10:40:58.3 |     10.000 |      0.000 |
+-----------+------------+------------+------------+
generator scan ['ec1f89aa'] (scan num: 12)





In [72]:
last_run = db[uid]

In [90]:
last_run.start

{'uid': 'ec1f89aa-e10f-463e-8d91-e97e0a9e76f7',
 'time': 1725975658.226195,
 'versions': {'ophyd': '1.9.0', 'bluesky': '1.12.0'},
 'scan_id': 12,
 'plan_type': 'generator',
 'plan_name': 'scan',
 'detectors': ['det1'],
 'motors': ['motor1'],
 'num_points': 6,
 'num_intervals': 5,
 'plan_args': {'detectors': ["SynGauss(prefix='', name='det1', read_attrs=['val'], configuration_attrs=['Imax', 'center', 'sigma', 'noise', 'noise_multiplier'])"],
  'num': 6,
  'args': ["SynAxis(prefix='', name='motor1', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration'])",
   0,
   10],
  'per_step': 'None'},
 'hints': {'dimensions': [[['motor1'], 'primary']]},
 'plan_pattern': 'inner_product',
 'plan_pattern_module': 'bluesky.plan_patterns',
 'plan_pattern_args': {'num': 6,
  'args': ["SynAxis(prefix='', name='motor1', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration'])",
   0,
   10]}}

### Eventos
Vão guardar informações de um ou mais eventos:

In [105]:
for event in last_run.events():
    display(event)
    break

{'descriptor': '5fed8be9-d522-498a-9fe3-6df176d1d51b',
 'uid': 'b1464a2c-38e9-4491-8037-8718ffdcbbb6',
 'time': 1725975658.2601922,
 'seq_num': 1,
 'data': {'det1': 5.0, 'motor1': 0.0, 'motor1_setpoint': 0.0},
 'timestamps': {'det1': 1725975658.240192,
  'motor1': 1725975658.2361913,
  'motor1_setpoint': 1725975658.2331889},
 'filled': {}}

In [106]:
pprint(event.to_name_dict_pair())

C:\Users\HOME\anaconda\Lib\site-packages\databroker\utils.py:208: UserWarning: Dot access may be removed in a future version. Use ['to_name_dict_pair'] instead of .to_name_dict_pair
  warnings.warn("Dot access may be removed in a future version. "
C:\Users\HOME\anaconda\Lib\site-packages\databroker\utils.py:211: UserWarning: In a future version of databroker, plain dicts without a '_name' attribute may be returned. Do not rely on '_name'.
  warnings.warn("In a future version of databroker, plain dicts "


('Event',
 {'data': {'det1': 5.0, 'motor1': 0.0, 'motor1_setpoint': 0.0},
  'descriptor': '5fed8be9-d522-498a-9fe3-6df176d1d51b',
  'filled': {},
  'seq_num': 1,
  'time': 1725975658.2601922,
  'timestamps': {'det1': 1725975658.240192,
                 'motor1': 1725975658.2361913,
                 'motor1_setpoint': 1725975658.2331889},
  'uid': 'b1464a2c-38e9-4491-8037-8718ffdcbbb6'})


### Metadata on Standard Plans

In [ ]:
(uid, ) = RE(scan([det1], motor1, -3, 3, 15, md={
    "motive": "Calibration",
    "sample": "Standard Sample"
}))
db[uid]

In [ ]:
last_run = db[-1]
last_run

### Start Event

- time: Horário de início do scan

- plan_name: Nome do plano executado, como 'scan' ou 'grid_scan'

- uid: ID único que identifica o scan realizado

- scan_id: Número inteiro amigável ao usuário, mas não necessáriamente único 

- Outros metadados inseridos pelo usuário ou plano

In [ ]:
last_run.start

### Stop Event

- time: Horário de fim do scan

- exit_status: Status do plano ao final ('abort', 'fail', 'success')

In [ ]:
last_run.stop

### Descriptors

In [ ]:
last_run.descriptors[0]

### Events

In [ ]:
last_run.table()

## Metadata Plan Stubs

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1].table()

In [ ]:
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    
    # Plan procedures
    yield from trigger_and_read([noisy_det])
    yield from rd(motor1)
    
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det', 'motor1']))
db[-1]

In [ ]:
def plan_with_data():
    yield from open_run(md={})
    yield from create(name="custom_data_stream")
    
    # Plan procedures
    yield from trigger(noisy_det)
    yield from rd(noisy_det)
    yield from rd(motor1)

    yield from save()
    yield from close_run()

uid = RE(plan_with_data(), LiveTable(['noisy_det']))
db[-1]

In [ ]:
db[-1].table("custom_data_stream")

create: Bundle future readings into a new Event document.

save: Close a bundle of readings and emit a completed Event document.

drop: Drop a bundle of readings without emitting a completed Event document.

subscribe: Subscribe the stream of emitted documents.

unsubscribe: Remove a subscription.


Plan Stubs

trigger: Trigger and acquisition.

read: Take a reading and add it to the current bundle of readings.

rd: Reads a single-value non-triggered object

Preprocessors -> What is a decorator

relative_set_decorator: Interpret 'set' messages on devices as relative to initial position.

reset_positions_decorator: Return movable devices to their initial positions at the end.

run_decorator: Enclose in 'open_run' and 'close_run' messages.

stage_decorator: 'Stage' devices (i.e., prepare them for use, 'arm' them) and then unstage.

finalize_decorator: try...finally helper

Suplemental Data



baseline_wrapper()

monitor_during_wrapper()

fly_during_wrapper()